<a href="https://colab.research.google.com/github/Muhammadsulton1/ML/blob/main/DL_mouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
with zipfile.ZipFile('/content/data.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('data')

In [ ]:
import os

In [ ]:
print('After zip extraction:')
print(os.listdir("data"))

After zip extraction:
['data']


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

In [ ]:
from torchsummary import summary
net = Net()

batch = []

#batch = batch(next(iter(trainloader)))

#summary(net.cuda(), (1,100, 100))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model = Net()

minimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

lr_decay = torch.optim.lr_scheduler.ExponentialLR(optimizer = minimizer, gamma = 0.95)

print("male weight:", gender_weights[1])

criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.as_tensor([gender_weights[1]]))

male weight: 0.5665193370165745


In [ ]:
from skimage.transform import resize 

In [ ]:
import numpy as np
import torch
from torch.autograd import Variable as var
from PIL import Image
import scipy.misc
import skimage
import random

image_width = 100
image_height = 100

np.random.seed(0)
    
# initialize train data
train_data = np.genfromtxt("features.csv", delimiter = ',')
np.random.shuffle(train_data) 

cross_validation_chunks = 10
test_chunk = 0

test_amount = float(len(train_data)) / cross_validation_chunks
train_cv_data = np.concatenate((train_data[:int(test_chunk * test_amount), :], train_data[int((test_chunk + 1) * test_amount):, :]))
test_cv_data = train_data[int(test_chunk * test_amount):int((test_chunk + 1) * test_amount)]        

class SpectrogramDataset(torch.utils.data.Dataset):
    
    def __init__(self, dir, data, augment = False):
        self.dir = dir
        self.data = data
        self.augment = augment

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):

        id = self.data[idx, 0]
        
        fn =  self.dir + "/" + "data" + (str(int(id)).zfill(9)) + "r.png"

        gray8 = np.array(Image.open(fn).convert('L'))
    
        gray8 = gray8.astype(np.float32)

        gray8_orig = np.copy(gray8)
                
        if self.augment:
            gray8 /= 255.
            max_pad_coeff = 0.1
            max_pad = int(gray8.shape[1] * max_pad_coeff)
            padded = np.zeros((gray8.shape[0], gray8.shape[1] + 2*max_pad), dtype = gray8.dtype)
            # converts to float
            padded = skimage.util.random_noise(padded, mode = 'gaussian', var = 0.01)
            padded[:, max_pad:(max_pad + gray8.shape[1])] = gray8
            gray8 = padded
            left = int((random.random() * 1 * max_pad))
            right = gray8.shape[1] - int((random.random() * 1 * max_pad))
            gray8 = gray8[:, left:right]
            # change volume
            aug_range = 0.5
            rc = 1.0 + aug_range - 2 * aug_range * random.random();            
            gray8 *= rc
            gray8[gray8 > 1.0] = 1.0
            # add noise
            # converts to 0..1, float
            gray8 = skimage.util.random_noise(gray8, mode = 'gaussian', var = 0.01 * random.random())
            gray8[gray8 > 1.0] = 1.0
            gray8 *= 255


        shape = gray8.shape

        max_duration = 100
    
        resized = np.zeros((233, max_duration), dtype = np.float32)

        max_width = min(shape[1], max_duration)

        resized[0:233, 0:max_width] = gray8[:, 0:max_width]

        # resize rescales the image if it's not uint8!

        resized = resized.astype(np.uint8)
        #resized = scipy.misc.imresize(resized, (image_height, image_width), interp = 'bilinear')
        resized = resize(resized, (image_height, image_width))

        resized = resized.astype(np.float32)
        
        
        # test how resulting images look like
        if self.augment:
        
            shape = gray8_orig.shape
        
            resized_orig = np.zeros((233, max_duration), dtype = np.float32)

            max_width = min(shape[1], max_duration)

            resized_orig[0:233, 0:max_width] = gray8_orig[:, 0:max_width]

            # resize rescales the image if it's not uint8!

            resized_orig = resized_orig.astype(np.uint8)
            #resized_orig = scipy.misc.imresize(resized_orig, (image_height, image_width), interp = 'bilinear')
            resized_orig = resize(resized_orig, (image_height, image_width))

            resized_orig = resized_orig.astype(np.float32)
        
            im = Image.fromarray(resized_orig).convert('L')
            im.save("data" + (str(int(id)).zfill(9)) + "_orig.png")
            im = Image.fromarray(resized).convert('L')
            im.save("data" + (str(int(id)).zfill(9)) + ".png")
        
        
        # hint: do we need to flatten if using pytorch conv2d?
        resized = resized.reshape(-1,100,100)
        
        return (resized, self.data[idx, 2])        



dataset = SpectrogramDataset("/content/data/data", train_cv_data, augment = True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size = 64)
test_dataset = SpectrogramDataset("/content/data/data", test_cv_data, augment = False)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)


gender_weights = []
for i in range(2):
    gender_weights.append(len(train_cv_data[train_cv_data[:, 2] == i])) 
    
gender_weights = 1.0 / (np.array(gender_weights, dtype = np.float) / len(train_cv_data))
gender_weights = gender_weights / np.sum(gender_weights) 
    
print("gender weights:", gender_weights)    

class Net(torch.nn.Module):
    def __init__(self):
      super(Net, self).__init__()

      self.conv1 = torch.nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2)
      self.act1 = torch.nn.ReLU()
      self.pool1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

      self.conv2 = torch.nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,padding=0)
      self.act2 = torch.nn.ReLU()
      self.pool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)


      #self.fc1 = torch.nn.Linear(image_width * image_height, 128)
      self.fc1 = torch.nn.Linear(16*23*23, 128)
      self.bn1 = torch.nn.BatchNorm1d(128)
      self.act3 = torch.nn.ReLU()

      self.fc2 = torch.nn.Linear(128, 128)      
      self.bn2 = torch.nn.BatchNorm1d(128)
      self.act4 = torch.nn.ReLU()
      self.fc3 = torch.nn.Linear(128, 1) 
      self.dropout = torch.nn.Dropout(0.3)      

      # hint: for conv net use torch.nn.Conv2d
      # hint: for conv net use torch.nn.BatchNorm1d(128)

    def forward(self, x):

      # hint : add conv layers here
      x = self.conv1(x)
      x = self.act1(x)
      x = self.pool1(x)

      x = self.conv2(x)
      x = self.act2(x)
      x = self.pool2(x)

      #print(x.shape) #для определения сколько параметров подавать на вход полносвязной сети
      x = x.view(-1,16*23*23)

      x = self.fc1(x)
      x = self.bn1(x)
      #x = torch.nn.functional.relu(x)
      x = self.dropout(x)

      x = self.fc2(x)
      x = self.bn2(x)
      #x = torch.nn.functional.relu(x) 
      x = self.dropout(x)

      x = self.fc3(x)

      return x      


model = Net()

#print(model)

minimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

lr_decay = torch.optim.lr_scheduler.ExponentialLR(optimizer = minimizer, gamma = 0.95)

print("male weight:", gender_weights[1])

criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.as_tensor([gender_weights[1]]))

def test_accuracy():

    correct_males = 0
    correct_females = 0
    total_males = 0
    total_females = 0

    for (test_data_tensor, test_targets_tensor) in test_data_loader:
        

        out = model(test_data_tensor)
        s_out = torch.round(torch.sigmoid(out))
        
        females = s_out[test_targets_tensor == 0]
        correct_females += len(females) - females.sum()
        total_females += len(females)

        males = s_out[test_targets_tensor == 1]
        correct_males += males.sum()
        total_males += len(males)

    ma = float(correct_males) / total_males
    fa = float(correct_females) / total_females
    print("males accuracy", ma)        
    print("females accuracy", fa)        
    print("balanced accuracy", (ma + fa) / 2 )        


test_accuracy()

epochs = 100
for epoch in range(int(epochs)):

    print("training epoch:", epoch + 1)

    for i, (data_tensor, targets_tensor) in enumerate(data_loader):

        minimizer.zero_grad()
        
        # compute prediction for the training data
        out = model(data_tensor)
                
        # compute loss value
        loss = criterion(out.squeeze(), targets_tensor)
        
        # compute the gradient
        loss.backward()
        
        # apply gradient values to the weights
        minimizer.step()

        accuracies = []

    test_accuracy()
    
  
    lr_decay.step()

    print("loss:", loss.item())
    print("learning rate", minimizer.param_groups[0]['lr'])

gender weights: [0.43348066 0.56651934]
male weight: 0.5665193370165745
males accuracy 0.3863080684596577
females accuracy 0.575503355704698
balanced accuracy 0.48090571208217786
training epoch: 1
males accuracy 0.2176039119804401
females accuracy 0.9580536912751678
balanced accuracy 0.5878288016278039
loss: 0.4615191163002963
learning rate 0.0095
training epoch: 2
males accuracy 0.3422982885085575
females accuracy 0.9161073825503355
balanced accuracy 0.6292028355294466
loss: 0.4204768038067066
learning rate 0.009025
training epoch: 3
males accuracy 0.34474327628361856
females accuracy 0.9429530201342282
balanced accuracy 0.6438481482089233
loss: 0.4032488126684846
learning rate 0.00857375
training epoch: 4
males accuracy 0.4669926650366748
females accuracy 0.9228187919463087
balanced accuracy 0.6949057284914917
loss: 0.3649808311817041
learning rate 0.0081450625
training epoch: 5
males accuracy 0.5134474327628362
females accuracy 0.8791946308724832
balanced accuracy 0.6963210318176597

KeyboardInterrupt: ignored